In [41]:
import warnings
warnings.simplefilter(action='ignore')

import tensorflow as tf
import pandas as pd
import numpy as np
import time
import itertools
from pprint import pprint as pp
import multiprocessing

import xgboost
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import load_data
from matplotlib import pyplot as plt

from bokeh.models import Jitter
from bokeh.layouts import column, row
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import FuncTickFormatter
from bokeh import palettes

%matplotlib inline
output_notebook()

Loading BokehJS ...

In [9]:
def fit_lr_model(X_train, y_train, X_val, y_val, label, fold_num, params):
    clf = LogisticRegression(**params, solver="saga", multi_class="multinomial")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    params["accuracy"] = accuracy_score(y_val, y_pred)
    params["precision"] = precision_score(y_val, y_pred, average="weighted")
    params["recall"] = recall_score(y_val, y_pred, average="weighted")
    params["f1"] = f1_score(y_val, y_pred, average="weighted")
    params["fold_num"] = fold_num
    params["label"] = label
    return params


def parameter_search_cv_lr(data, params_dict_list, random_state=0):
    rows = []
    for fold_num, (train, val) in enumerate(data.cv):
        print(fold_num, end=",")
        t0 = time.time()
        for label in ["gender", "tumor", "tissue"]:
            print(label, end=",")
            X_train, y_train = train.STL[label]["X"], train.STL[label]["y"]
            X_val, y_val = val.STL[label]["X"], val.STL[label]["y"]            
            assert((X_train.shape[0] == y_train.shape[0]) 
                   and (X_val.shape[0] == y_val.shape[0]))
            args_list = [[X_train, y_train, X_val, y_val, label, fold_num, i] 
                             for i in params_dict_list]
            p = multiprocessing.Pool(16)
            rows += p.starmap(fit_lr_model, args_list)
            p.terminate()
        print("it takes {0} seconds to run fold {1}".format(time.time()-t0, fold_num))
    result_df = pd.DataFrame(rows)
    result_df = result_df.replace(np.nan, "unbalanced")
    return result_df

### Predictions

In [8]:
params_list = list(itertools.product(["balanced", None], 
                                     ["l1", "l2"], 
                                     [0.01, 0.1, 1, 10, 100]))
params_dict_list = [dict(zip(["class_weight", "penalty", "C"], i)) for i in params_list]

for var in [0.2, 0.6, 0.7, 0.8, 0.9]:
    print("PCA variance:", var)
    data = load_data.read_data_sets("./data/mRNA_PCA_{0}_variance_StandardScaled.csv".format(var),
                                    random_state=0)
    result_df = parameter_search_cv_lr(data, params_dict_list, random_state=0)
    result_df["PCA"] = var
    result_df.to_csv("./results/LR/PCA_{0}_LR_gridsearch.csv".format(var), index=None)

PCA variance: 0.2
0,gender,tumor,tissue,it takes 3.9861152172088623 seconds to run fold 0
1,gender,tumor,tissue,it takes 3.9794657230377197 seconds to run fold 1
2,gender,tumor,tissue,it takes 3.877807855606079 seconds to run fold 2
3,gender,tumor,tissue,it takes 3.9850282669067383 seconds to run fold 3
4,gender,tumor,tissue,it takes 3.973231077194214 seconds to run fold 4


In [12]:
filename = "./results/LR/PCA_{0}_LR_gridsearch.csv"
result_df = pd.concat([pd.read_csv(filename.format(i)) for i in [0.2, 0.6, 0.7, 0.8, 0.9]])

In [13]:
## Various plot related functions and attributes
colors = ["red", "olive", "goldenrod", "skyblue", "orange", "salmon"]

def x_ticker_name():
    name_dict = {0:"PCA_0.6", 1:"PCA_0.7", 2:"PCA_0.8", 3:"PCA_0.9"}
    return name_dict[tick]

### Compare between difference dataset

In [14]:
plots = []
for label in ["gender", "tissue", "tumor"]:
    p = figure(plot_width=300, plot_height=500, title=label)
    for i, var in enumerate([0.6, 0.7, 0.8, 0.9]):
        y = result_df[(result_df['PCA'] == var) & (result_df["label"]==label)]['accuracy']
        p.circle(x={'value': i, 'transform': Jitter(width=0.3)}, y=y, color=colors[i], size=2)
    p.xaxis.ticker = [0, 1, 2, 3]
    p.xaxis.formatter = FuncTickFormatter.from_py_func(x_ticker_name)
    plots.append(p)
show(row(plots))

### Compare between balanced vs unbalanced loss function

In [15]:
plots = []
for label in ["gender", "tissue", "tumor"]:
    p = figure(plot_width=300, plot_height=500, title=label)
    for i, var in enumerate([0.6, 0.7, 0.8, 0.9]):
        y = result_df[(result_df['PCA'] == var) & (result_df["label"]==label)]
        balanced = y[y.class_weight=="balanced"]["accuracy"]
        unbalanced = y[y.class_weight=="unbalanced"]["accuracy"]
        p.circle(x={'value': i, 'transform': Jitter(width=0.2)}, 
                 y=balanced, color="grey", size=2, legend="balanced")
        p.circle(x={'value': i, 'transform': Jitter(width=0.2)}, 
                 y=unbalanced, color="red", size=2, legend="unbalanced")
    p.xaxis.ticker = [0, 1, 2, 3]
    p.xaxis.formatter = FuncTickFormatter.from_py_func(x_ticker_name)
    p.legend.location="bottom_right"
    plots.append(p)
show(row(plots))

### Compare between l1 and l2 (with regularization strength colored)

In [16]:
def transform_C(Cs, palette):
    color_idx = (np.log10(Cs) + 2).astype("int")
    return np.array(palette)[color_idx]
plots = []
for label in ["gender", "tissue", "tumor"]:
    p = figure(plot_width=600, plot_height=400, title=label)
    for i, var in enumerate([0.6, 0.7, 0.8, 0.9]):
        y = result_df[(result_df['PCA'] == var) & (result_df["label"]==label)]
        l1 = y[y.penalty=="l1"]["accuracy"]
        l2 = y[y.penalty=="l2"]["accuracy"]
        colors_l1 = transform_C(y[y.penalty=="l1"]["C"], palettes.Blues6)
        colors_l2 = transform_C(y[y.penalty=="l2"]["C"], palettes.Reds6)
        p.circle(x={'value': i-0.11, 'transform': Jitter(width=0.2)}, 
                 y=l1, size=3, legend="L1", color=colors_l1)
        p.circle(x={'value': i+0.11, 'transform': Jitter(width=0.2)}, 
                 y=l2, size=3, legend="L2", color=colors_l2)
    p.xaxis.ticker = [0, 1, 2, 3]
    p.xaxis.formatter = FuncTickFormatter.from_py_func(x_ticker_name)
    p.legend.location="bottom_right"
    plots.append(p)
show(column(plots))

### Compare between l1 and l2 (with regularization strength colored) removed balanced

In [17]:
plots = []
for label in ["gender", "tissue", "tumor"]:
    p = figure(plot_width=600, plot_height=400, title=label)
    for i, var in enumerate([0.6, 0.7, 0.8, 0.9]):
        y = result_df[(result_df['PCA'] == var) & (result_df["label"]==label) & 
                      (result_df["class_weight"]=="unbalanced")]
        l1 = y[y.penalty=="l1"]["accuracy"]
        l2 = y[y.penalty=="l2"]["accuracy"]
        colors_l1 = transform_C(y[y.penalty=="l1"]["C"], palettes.Blues6)
        colors_l2 = transform_C(y[y.penalty=="l2"]["C"], palettes.Reds6)
        p.circle(x={'value': i-0.11, 'transform': Jitter(width=0.2)}, 
                 y=l1, size=3, legend="L1", color=colors_l1)
        p.circle(x={'value': i+0.11, 'transform': Jitter(width=0.2)}, 
                 y=l2, size=3, legend="L2", color=colors_l2)
    p.xaxis.ticker = [0, 1, 2, 3]
    p.xaxis.formatter = FuncTickFormatter.from_py_func(x_ticker_name)
    p.legend.location="bottom_right"
    plots.append(p)
show(column(plots))

### Best results

In [117]:
fold_avg = result_df.groupby(["PCA", "label", "class_weight", "penalty", "C"]).agg(
    [np.mean, np.std])[["accuracy", "f1", "precision", "recall"]]
fold_avg.to_hdf("./results/LR/LR_grid_search_all.hdf", "LR")

In [179]:
df = pd.read_hdf("./results/LR/LR_grid_search_accuracy.hdf")

In [260]:
temp_df = df.unstack().swaplevel(i=0,j=2, axis=1).stack().stack().reset_index(level=[0,1,2])
best_result = temp_df[["mean"]].max(level=[0,1]).reset_index().set_index("mean")
assert(best_result.shape[0] == len(set(best_result.index)))
best_result = best_result.join(temp_df.reset_index().set_index("mean"), rsuffix="x").reset_index().set_index(
    ["PCA", "label"]).sort_index()[["mean", "std", "class_weight", "penalty" ,"C"]]

In [262]:
best_result.to_hdf("./results/LR/LR_best_accuracy.hdf", "best")

In [263]:
best_result

mean       std class_weight penalty       C
PCA label                                                  
0.2 gender  0.530736  0.012651   unbalanced      l1    0.10
    tissue  0.308936  0.003198   unbalanced      l1  100.00
    tumor   0.876723  0.003155   unbalanced      l1    1.00
    tumor   0.876723  0.003155   unbalanced      l2    1.00
0.6 gender  0.706645  0.006428     balanced      l1    0.01
    tissue  0.914166  0.002693   unbalanced      l1    1.00
    tumor   0.966746  0.001912   unbalanced      l1    1.00
0.7 gender  0.738418  0.004629     balanced      l2    0.10
    tissue  0.936564  0.004538   unbalanced      l2    0.10
    tumor   0.971481  0.002060   unbalanced      l1    1.00
0.8 gender  0.853647  0.007879   unbalanced      l2    1.00
    tissue  0.951154  0.002893   unbalanced      l2    0.10
    tumor   0.977079  0.001994   unbalanced      l1    0.10
0.9 gender  0.992673  0.001655   unbalanced      l2    0.01
    tissue  0.967867  0.003028   unbalanced      l1  100.00
    tumor   0.978789  0.002354   unbalanced      l1    1.00

### Predict Age with linear regression

In [59]:
def fit_EN(X_train, y_train, X_val, y_val, fold_num, params):
    clf = ElasticNet(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    results = params.copy()
    results["mse"] = mean_squared_error(y_val, y_pred)
    results["mae"] = mean_absolute_error(y_val, y_pred)
    results["r2"] = r2_score(y_val, y_pred)
    results["fold_num"] = fold_num
    return results


def parameter_search_cv_EN(data, params_dict_list, random_state=0):
    rows = []
    for fold_num, (train, val) in enumerate(data.cv):
        print(fold_num, end=",")
        t0 = time.time()
        X_train, y_train = train.STL["age"]["X"], train.STL["age"]["y"]
        X_val, y_val = val.STL["age"]["X"], val.STL["age"]["y"]            
        assert((X_train.shape[0] == y_train.shape[0]) 
               and (X_val.shape[0] == y_val.shape[0]))
        args_list = [[X_train, y_train, X_val, y_val, fold_num, i] 
                     for i in params_dict_list]
        p = multiprocessing.Pool(16)
        rows += p.starmap(fit_EN, args_list)
        p.terminate()
        print("it takes {0} seconds to run fold {1}".format(time.time()-t0, fold_num))
    result_df = pd.DataFrame(rows)
    return result_df

In [ ]:
params_list = list(itertools.product([0.01, 0.1, 1, 10, 100], 
                                     [0, 0.5, 1],
                                     ))
params_dict_list = [dict(zip(["alpha", "l1_ratio"], i)) 
                    for i in params_list]
for var in [0.2, 0.6, 0.7, 0.8, 0.9]:
    print("PCA variance:", var)
    data = load_data.read_data_sets("./data/mRNA_PCA_{0}_variance_StandardScaled.csv".format(var),
                                    random_state=0)
    result_df = parameter_search_cv_EN(data, 
        params_dict_list, random_state=0)
    result_df["PCA"] = var
    result_df.to_csv("./results/LR/PCA_{0}_Age_ElasticNet_gridsearch.csv".format(var), index=None)

PCA variance: 0.2
0,it takes 0.5757005214691162 seconds to run fold 0
1,it takes 0.6735312938690186 seconds to run fold 1
2,it takes 0.6706137657165527 seconds to run fold 2
3,it takes 0.7616991996765137 seconds to run fold 3
4,it takes 0.6685945987701416 seconds to run fold 4
PCA variance: 0.6
0,it takes 1.2110881805419922 seconds to run fold 0
1,it takes 1.3993170261383057 seconds to run fold 1
2,it takes 1.508702039718628 seconds to run fold 2
3,it takes 1.306990623474121 seconds to run fold 3
4,it takes 1.4162983894348145 seconds to run fold 4
PCA variance: 0.7
0,it takes 1.7881245613098145 seconds to run fold 0
1,it takes 2.122418165206909 seconds to run fold 1
2,it takes 2.0416510105133057 seconds to run fold 2
3,it takes 2.144395112991333 seconds to run fold 3
4,it takes 2.0372986793518066 seconds to run fold 4
PCA variance: 0.8
0,it takes 4.9278035163879395 seconds to run fold 0
1,it takes 5.660952568054199 seconds to run fold 1
2,it takes 6.19904088973999 seconds to run fold 2